In [219]:
def setenceToTokens(sentence):
    return sentence.split()
def tokenToWordAndTag(token):
#     print(token)
    word, tag = token.split('/')
    return (word, tag)
def tokensToPairs(tokens):
    return [tokenToWordAndTag(token) for token in tokens]
def tokensToWordsAndTags(tokens):
    x, y = [], []
    for token in tokens:
        word, tag = tokenToWordAndTag(token)
        x.append(word)
        y.append(tag)
    return x, y
def process(data):
    """
    data: list of senteces
    """
    x, y = [], []
    for sentence in data:
        words, tags = tokensToWordsAndTags(setenceToTokens(sentence))
        x.append(words)
        y.append(tags)
    return x, y
def setUnknown(x, vocab):
    for words in x:
        for i in range(len(words)):
            if words[i] not in vocab:
                words[i] = '<UNK>'
    return x

## POS tagging
### (a) Preprocessing

In [245]:
# Load packages
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import nltk

# Load data
trn = open("data/trn.pos").read().strip().split("\n")
print("Training data ...")
print("{}".format(len(trn)))
# print(trn[:3])
pairs = [tokenToWordAndTag(token) for token in setenceToTokens(trn[0])]
# print(pairs)
tokenizedSetences = [setenceToTokens(setence) for setence in trn]
pairLists = [tokensToPairs(tokens) for tokens in tokenizedSetences]
# print(pairLists[0])
count = Counter()
vocab = set()
for pairList in pairLists:
    for pair in pairList:
        count[pair[0]] += 1
        vocab.add(pair[0])
# print(pairLists[0])
for pairList in pairLists:
    for i, pair in enumerate(pairList):
        if count[pair[0]] < 2:
            pairList[i] = ('<UNK>', pair[1])
            count['<UNK>'] += 1
            if pair[0] in vocab:
                vocab.remove(pair[0])
count['<start>'] = len(pairLists)
count['<end>'] = len(pairLists)

Training data ...
54097


### (b)  Estimating transition probability

In [247]:
transitionCount = Counter()
for pairList in pairLists:
    transitionCount[('<start>', pairList[0][1])] += 1
    transitionCount[(pairList[-1][1], '<end>')] += 1
    for i in range(1, len(pairList)):
        transitionCount[(pairList[i-1][1], pairList[i][1])] += 1
tagCount = Counter()
stateSpace = set()
vocab = set()
for pairList in pairLists:
    for pair in pairList:
        tagCount[pair[1]] += 1
        stateSpace.add(pair[1])
        vocab.add(pair[0])
stateSpace = list(stateSpace)
tagCount['<start>'] = len(pairLists)
tagCount['<end>'] = len(pairLists)
# print(transitionCount)

In [248]:
def calculateTransitionProb(transitionCount, stateSpace, alpha = 1):
    transitionProb = {}
    for tag1 in stateSpace+['<start>']:
        for tag2 in stateSpace+['<end>']:
            transitionProb[(tag1, tag2)] = (transitionCount[(tag1, tag2)] + alpha) / (tagCount[tag1] + alpha*len(stateSpace))
    return transitionProb

In [250]:
transitionProb = calculateTransitionProb(transitionCount, stateSpace, 1)
# print(transitionProb)

In [251]:
testSum = 0
for key, value in transitionProb.items():
    if key[0] == 'C':
        testSum += value
print(testSum)

1.0000076051410753


### (c) Estimating the emission probability

In [252]:
emissionCount = Counter()
beta = 1
stateSpace = set()
vocab = set()
for pairList in pairLists:
    for pair in pairList:
        emissionCount[(pair[1], pair[0])] += 1
        stateSpace.add(pair[1])
        vocab.add(pair[0])
stateSpace = list(stateSpace)
def calculateEmissionProb(emissionCount, stateSpace, beta):    
    emissionProb = {}
    for tag in stateSpace:
#         print(tag)
        for word in vocab:
            emissionProb[(tag, word)] = (emissionCount[(tag, word)] + beta)/(tagCount[tag] + beta*len(vocab))
    return emissionProb
emissionProb = calculateEmissionProb(emissionCount, stateSpace, beta)

In [253]:
testSum = 0
for key, value in emissionProb.items():
    if key[0] == 'N':
        testSum += value
print(testSum)

0.999999999999972


### (d) The Viterbi Algorithm

In [254]:
# Convert to log probabilities
from math import log
def convertToLog(prob):
    for k in prob:
        prob[k] = log(prob[k])
    return prob
logTransitionProb = convertToLog(transitionProb)
logEmissionProb = convertToLog(emissionProb)

In [258]:
import numpy as np
def viterbi(x, logTransitionProb, logEmissionProb):
    """
    Return the MAP estimate of y.
    Parameters
    ----------
    x : array (T,)
        Observation state sequence. int dtype.
    logTransitionProb : dictionary
        State transition map.
        details.
    logEmissionProb : dictionary
        Emission map.

    Returns
    -------
    y : array (T,)
        Maximum a posteriori probability estimate of hidden state trajectory.
    """
    # Cardinality of the state space
    K = len(stateSpace)
    T = len(x)
    maxProb = np.empty((T, K), np.float64)
    bestPrev = np.empty((T, K), dtype=object)

    # Initilaize the tracking tables from first observation
    for i, tag in enumerate(stateSpace):
        maxProb[0, i] = logTransitionProb[('<start>', tag)] + logEmissionProb[(tag, x[0])]
    bestPrev[0, :] = '<start>'
    for i in range(1, len(x)):
        for j, tag in enumerate(stateSpace):
            bestPrevProb, bestPrevTag = float('-inf'), None
            for k, prevTag in enumerate(stateSpace):
                temp = maxProb[i-1, k] + logTransitionProb[(prevTag, tag)] + logEmissionProb[(tag, x[i])]
                if temp > bestPrevProb:
                    bestPrevProb = temp
                    bestPrevTag = prevTag
            maxProb[i, j] = bestPrevProb
            bestPrev[i, j] = bestPrevTag
    bestLastProb, betLastTag = float('-inf'), None
    for i, prevTag in enumerate(stateSpace):
        temp = maxProb[-1, i] + logTransitionProb[(prevTag, '<end>')]
        if temp > bestLastProb:
            bestLastProb = temp
            bestLastTag = prevTag
    y = [bestLastTag]
    for i in reversed(range(1, T)):
#         print(i)
        y.append(bestPrev[i, stateSpace.index(bestLastTag)])
        bestLastTag = y[-1]
    return y[::-1]

In [259]:
def pairListToWords(pairList):
    return [pair[0] for pair in pairList]

In [260]:
def calculateAccuracy(x, y, logTransitionProb, logEmissionProb):
    correct, total = 0, 0
    for words, tags in zip(x, y):
        predicted_tags = viterbi(words, logTransitionProb, logEmissionProb)
        for predicted_tag, tag in zip(predicted_tags, tags):
            total += 1
            if predicted_tag == tag:
                correct += 1
    return correct/total

In [262]:
## load dev
dev = open("data/dev.pos").read().strip().split("\n")
x, y = process(dev)
x = setUnknown(x, vocab)
# print(x[:3])
# print(y[:3])
print('Accuracy: {}'.format(calculateAccuracy(x, y, logTransitionProb, logEmissionProb)))

Accuracy: 0.9474509398528627


### (e) Tuning $\alpha$ and $\beta$

In [237]:
for alpha in [1,100, 200 , 500, 1000, 2000, 3000, 5000, 10000, 20000]:
    transitionProb = calculateTransitionProb(transitionCount, stateSpace, alpha)
    logTransitionProb = convertToLog(transitionProb)
    print('alpha={}, accuracy={}'.format(alpha, calculateAccuracy(x, y, logTransitionProb, logEmissionProb)))

alpha=1, accuracy=0.9474509398528627
alpha=100, accuracy=0.9478213676538998
alpha=200, accuracy=0.9480195034544546
alpha=500, accuracy=0.9482865560552024
alpha=1000, accuracy=0.9489671094571079
alpha=2000, accuracy=0.9497941110594235
alpha=3000, accuracy=0.9498199548594959
alpha=5000, accuracy=0.9502248410606295
alpha=10000, accuracy=0.9496735066590858
alpha=20000, accuracy=0.9472355748522596


In [243]:
transitionProb = calculateTransitionProb(transitionCount, stateSpace, 5000)
logTransitionProb = convertToLog(transitionProb)
for beta in [1, 2, 3, 4, 5]:
    emissionProb = calculateEmissionProb(emissionCount, stateSpace, beta)
    logEmissionProb = convertToLog(emissionProb)
    print('beta={}, accuracy={}'.format(beta, calculateAccuracy(x, y, logTransitionProb, logEmissionProb)))

beta=1, accuracy=0.9502248410606295
beta=2, accuracy=0.9462448958494857
beta=3, accuracy=0.9422563360383177
beta=4, accuracy=0.9388794128288623
beta=5, accuracy=0.9361485846212161


### (f) Testing

In [264]:
tst = open("data/tst.pos").read().strip().split("\n")
x, y = process(tst)
x = setUnknown(x, vocab)
# alpha = 5000
transitionProb = calculateTransitionProb(transitionCount, stateSpace, 5000)
logTransitionProb = convertToLog(transitionProb)
# beta = 1
emissionProb = calculateEmissionProb(emissionCount, stateSpace, 1)
logEmissionProb = convertToLog(emissionProb)
print('Accuracy on testing data: {}'.format(calculateAccuracy(x, y, logTransitionProb, logEmissionProb)))

Accuracy on testing data: 0.9515465680318943
